In [50]:
import pandas as pd
from scipy.optimize import linprog
import numpy as np
from gurobipy import GRB
import gurobipy as gb
from gurobipy import Model, GRB, quicksum
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum


In [35]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 38.9 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
import gurobipy

In [ ]:
pip install gurobipy

In [25]:
df_direct_production_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Direct_Production_Facilities.csv')
df_transship_production_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Transship_Production_Facilities.csv')
df_transship_distribution_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Transship_Distribution_Centers.csv')
df_cost_production_to_refinement = pd.read_csv('/Users/youziya/Downloads/Cost_Production_to_Refinement.csv')
df_cost_production_to_transshipment = pd.read_csv('/Users/youziya/Downloads/Cost_Production_to_Transshipment.csv')
df_cost_transshipment_to_refinement = pd.read_csv('/Users/youziya/Downloads/Cost_Transshipment_to_Refinement.csv')
df_refinement_demand = pd.read_csv('/Users/youziya/Downloads/Refinement_Demand.csv')


In [ ]:
df_refinement_demand.info()

In [26]:
dfs = {
    'Direct Production Capacity': df_direct_production_capacity,
    'Transship Production Capacity': df_transship_production_capacity,
    'Transship Distribution Capacity': df_transship_distribution_capacity,
    'Cost Production to Refinement': df_cost_production_to_refinement,
    'Cost Production to Transshipment': df_cost_production_to_transshipment,
    'Cost Transshipment to Refinement': df_cost_transshipment_to_refinement,
    'Refinement Demand': df_refinement_demand
}

dfs


{'Direct Production Capacity':     ProductionFacility  Capacity
 0                    1       462
 1                    2       103
 2                    3       460
 3                    4       325
 4                    5       227
 5                    6       217
 6                    7       205
 7                    8       521
 8                    9       548
 9                   10       191
 10                  11       361
 11                  12       411
 12                  13       104
 13                  14       155
 14                  15       285
 15                  16       109
 16                  17       422
 17                  18       438
 18                  19       501
 19                  20       139
 20                  21       462
 21                  22       504
 22                  23       106
 23                  24       132
 24                  25       298,
 'Transship Production Capacity':     ProductionFacility  Capacity
 0                

In [47]:

# Create the optimization model
model = LpProblem("Can2Oil_Optimization", LpMinimize)

# Helper function to create a dictionary from DataFrame for easy access
def create_dict_from_df(df, index_cols, value_col):
    return {(row[index_cols[0]], row[index_cols[1]]): row[value_col] for _, row in df.iterrows()}

# Decision Variables
# x_ij: oil shipped directly from production facility i to refinement center j
# y_ik: oil shipped from transship production facility i to transshipment hub k
# z_kj: oil shipped from transshipment hub k to refinement center j

# Extracting facilities and centers
direct_prod_facilities = df_direct_production_capacity['ProductionFacility'].unique()
transship_prod_facilities = df_transship_production_capacity['ProductionFacility'].unique()
refinement_centers = df_refinement_demand['RefinementCenter'].unique()
transshipment_hubs = df_transship_distribution_capacity['TransshipmentHub'].unique()

# Creating variables
x = LpVariable.dicts("x", [(i, j) for i in direct_prod_facilities for j in refinement_centers], lowBound=0)
y = LpVariable.dicts("y", [(i, k) for i in transship_prod_facilities for k in transshipment_hubs], lowBound=0)
z = LpVariable.dicts("z", [(k, j) for k in transshipment_hubs for j in refinement_centers], lowBound=0)

# Costs dictionaries
cost_direct = create_dict_from_df(df_cost_production_to_refinement, ['ProductionFacility', 'RefinementCenter'], 'Cost')
cost_transship = create_dict_from_df(df_cost_production_to_transshipment, ['ProductionFacility', 'TransshipmentHub'], 'Cost')
cost_trans_to_refine = create_dict_from_df(df_cost_transshipment_to_refinement, ['TransshipmentHub', 'RefinementCenter'], 'Cost')

# Capacities dictionaries
capacity_direct = df_direct_production_capacity.set_index('ProductionFacility')['Capacity'].to_dict()
capacity_transship = df_transship_production_capacity.set_index('ProductionFacility')['Capacity'].to_dict()
capacity_transship_hub = df_transship_distribution_capacity.set_index('TransshipmentHub')['Capacity'].to_dict()
demand_refine = df_refinement_demand.set_index('RefinementCenter')['Demand'].to_dict()

# Objective Function
model += (lpSum([cost_direct[i, j] * x[i, j] for i in direct_prod_facilities for j in refinement_centers]) +
          lpSum([cost_transship[i, k] * y[i, k] for i in transship_prod_facilities for k in transshipment_hubs]) +
          lpSum([cost_trans_to_refine[k, j] * z[k, j] for k in transshipment_hubs for j in refinement_centers]))

# Constraints
# Production capacity constraints for direct production facilities
for i in direct_prod_facilities:
    model += lpSum([x[i, j] for j in refinement_centers]) <= capacity_direct[i]

# Production capacity constraints for transshipment production facilities
for i in transship_prod_facilities:
    model += lpSum([y[i, k] for k in transshipment_hubs]) <= capacity_transship[i]

# Transshipment hub capacity constraints
for k in transshipment_hubs:
    model += lpSum([y[i, k] for i in transship_prod_facilities]) <= capacity_transship_hub[k]
    model += lpSum([z[k, j] for j in refinement_centers]) <= capacity_transship_hub[k]

# Refinement center demand constraints
for j in refinement_centers:
    model += lpSum([x[i, j] for i in direct_prod_facilities]) + lpSum([z[k, j] for k in transshipment_hubs]) == demand_refine[j]

# Solve the model
model.solve()

# Output results
solution_status = model.status
total_cost = model.objective.value()

# Corrected output
print("Solution Status:", solution_status)
print("Total Minimized Cost:", total_cost)



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/5v/b37q3lz15lq75f7xfjzshd900000gn/T/53cbf74c9aab4e2da470763b65e8a23e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/5v/b37q3lz15lq75f7xfjzshd900000gn/T/53cbf74c9aab4e2da470763b65e8a23e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 550 RHS
At line 600 BOUNDS
At line 601 ENDATA
Problem MODEL has 49 rows, 165 columns and 330 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 32 (-17) rows, 135 (-30) columns and 270 (-60) elements
0  Obj 0 Primal inf 8728 (5)
31  Obj 22093.677 Primal inf 0.699993 (7)
38  Obj 22094.007
Optimal - objective value 22094.007
After Postsolve, objective 22094.007, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 22094.00717 - 38 iterations tim

### Using GRB

In [52]:
import gurobipy as gp
from gurobipy import GRB

# Assuming you have loaded the necessary data into the following variable

# Create a new model
model = gp.Model("Can2Oil_Transportation")

# Add decision variables
x = model.addVars(direct_prod_facilities, refinement_centers, name="x", vtype=GRB.CONTINUOUS, lb=0)
y = model.addVars(transship_prod_facilities, transshipment_hubs, name="y", vtype=GRB.CONTINUOUS, lb=0)
z = model.addVars(transshipment_hubs, refinement_centers, name="z", vtype=GRB.CONTINUOUS, lb=0)

# Objective function
model.setObjective(
    gp.quicksum(cost_direct[i, j] * x[i, j] for i in direct_prod_facilities for j in refinement_centers) +
    gp.quicksum(cost_transship[i, k] * y[i, k] for i in transship_prod_facilities for k in transshipment_hubs) +
    gp.quicksum(cost_trans_to_refine[k, j] * z[k, j] for k in transshipment_hubs for j in refinement_centers),
    GRB.MINIMIZE)

# Constraints
# Production capacity constraints for direct and transshipment production facilities
for i in direct_prod_facilities:
    model.addConstr(gp.quicksum(x[i, j] for j in refinement_centers) <= capacity_direct[i], f"DirectCap_{i}")
for i in transship_prod_facilities:
    model.addConstr(gp.quicksum(y[i, k] for k in transshipment_hubs) <= capacity_transship[i], f"TransshipProdCap_{i}")

# Transshipment hub capacity constraints
for k in transshipment_hubs:
    model.addConstr(gp.quicksum(y[i, k] for i in transship_prod_facilities) <= capacity_transship_hub[k], f"TransshipHubInCap_{k}")
    model.addConstr(gp.quicksum(z[k, j] for j in refinement_centers) <= capacity_transship_hub[k], f"TransshipHubOutCap_{k}")

# Demand constraints for refinement centers
for j in refinement_centers:
    model.addConstr(gp.quicksum(x[i, j] for i in direct_prod_facilities) + gp.quicksum(z[k, j] for k in transshipment_hubs) == demand_refine[j], f"Demand_{j}")

# Solve the model
model.optimize()

# Output the optimal transportation cost
if model.status == GRB.OPTIMAL:
    print(f"Optimal Transportation Cost: {model.objVal}")
else:
    print("Optimal solution was not found.")

# Output the optimal values for decision variables if needed
if model.status == GRB.OPTIMAL:
    optimal_values_x = { (i, j): x[i, j].X for i in direct_prod_facilities for j in refinement_centers }
    optimal_values_y = { (i, k): y[i, k].X for i in transship_prod_facilities for k in transshipment_hubs }
    optimal_values_z = { (k, j): z[k, j].X for k in transshipment_hubs for j in refinement_centers }

    print("Optimal Values of x_ij (Direct Shipments):", optimal_values_x)
    print("Optimal Values of y_ik (To Transshipment):", optimal_values_y)
    print("Optimal Values of z_kj (From Transshipment):", optimal_values_z)




Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D56)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 49 rows, 165 columns and 330 nonzeros
Model fingerprint: 0x6abe2a2f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 17 rows and 30 columns
Presolve time: 0.03s
Presolved: 32 rows, 135 columns, 270 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   1.009982e+03   0.000000e+00      0s
      31    2.2094007e+04   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.05 seconds (0.00 work units)
Optimal objective  2.209400717e+04
Optimal Transportation Cost: 22094.007168097058
Optimal Values of x_ij (Direct Shipments): {(1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 46

### Question 2

In [48]:
# After solving the model
model.solve()

# Extracting the optimal values of x_ij, y_ik, and z_kj
optimal_values_x = {(i, j): x[i, j].varValue for i in direct_prod_facilities for j in refinement_centers}
optimal_values_y = {(i, k): y[i, k].varValue for i in transship_prod_facilities for k in transshipment_hubs}
optimal_values_z = {(k, j): z[k, j].varValue for k in transshipment_hubs for j in refinement_centers}

# You can now print these values or use them in further calculations
print("Optimal Values of x_ij (Direct Shipments):", optimal_values_x)
print("Optimal Values of y_ik (To Transshipment):", optimal_values_y)
print("Optimal Values of z_kj (From Transshipment):", optimal_values_z)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/5v/b37q3lz15lq75f7xfjzshd900000gn/T/a323942f0c38488892641ab01815ab4b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/5v/b37q3lz15lq75f7xfjzshd900000gn/T/a323942f0c38488892641ab01815ab4b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 550 RHS
At line 600 BOUNDS
At line 601 ENDATA
Problem MODEL has 49 rows, 165 columns and 330 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 32 (-17) rows, 135 (-30) columns and 270 (-60) elements
0  Obj 0 Primal inf 8728 (5)
31  Obj 22093.677 Primal inf 0.699993 (7)
38  Obj 22094.007
Optimal - objective value 22094.007
After Postsolve, objective 22094.007, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 22094.00717 - 38 iterations tim

In [ ]:
# Assuming optimal_values_x, optimal_values_y, and optimal_values_z are dictionaries 
# containing the optimal values of x_ij, y_ik, and z_kj from the Gurobi solution.

# Calculate total transshipped oil
total_transshipped_oil = sum(optimal_values_y.values())

# Calculate total shipped oil (direct + transshipped)
total_shipped_oil = sum(optimal_values_x.values()) + total_transshipped_oil

# Calculate the proportion of transshipped oil
if total_shipped_oil > 0:
    proportion_transshipped = total_transshipped_oil / total_shipped_oil
else:
    proportion_transshipped = 0

proportion_transshipped


### Question 3
limits the total amount of oil transshipped. limit the transshipped oil to a certain proportion of the total shipped oil which is p

In [57]:
penalty_factor = 0.50 * 4.34  # 20% of the average direct cost

# Incorporate this penalty factor into the objective function
model.setObjective(
    gp.quicksum(cost_direct[i, j] * x[i, j] for i in direct_prod_facilities for j in refinement_centers) +
    gp.quicksum((cost_transship[i, k] + penalty_factor) * y[i, k] for i in transship_prod_facilities for k in transshipment_hubs) +
    gp.quicksum(cost_trans_to_refine[k, j] * z[k, j] for k in transshipment_hubs for j in refinement_centers),
    GRB.MINIMIZE)

# Solve the model
model.optimize()

# Output the optimal transportation cost and check the solution
if model.status == GRB.OPTIMAL:
    print(f"Optimal Transportation Cost: {model.objVal}")
else:
    print("Optimal solution was not found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D56)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 52 rows, 165 columns and 795 nonzeros
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2094007e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.209400717e+04
Optimal Transportation Cost: 22094.007168097058


In [59]:
# Example: Set a hard limit on the total amount of oil that can be transshipped
transshipment_limit = 10  # Define a suitable limit

# Add constraint to the model
total_transshipment = gp.quicksum(y[i, k] for i in transship_prod_facilities for k in transshipment_hubs)
model.addConstr(total_transshipment <= transshipment_limit, "TransshipmentUpperLimit")

# Solve the model
model.optimize()

# Output the optimal transportation cost
if model.status == GRB.OPTIMAL:
    print(f"Optimal Transportation Cost: {model.objVal}")
else:
    print("Optimal solution was not found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D56)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 54 rows, 165 columns and 855 nonzeros
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2094007e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.209400717e+04
Optimal Transportation Cost: 22094.007168097058


### Quetion 5
Which of the two modeling approaches would you recommend the company take to determine a transportation plan that reduces the amount of canola oil that is transshipped?

Between the two modeling approaches to reduce the amount of canola oil that is transshipped — adding a transshipment limit constraint or modifying the objective function to penalize transshipment — the choice depends on Can2Oil's strategic priorities and operational flexibility. 

1. Adding a Transshipment Limit Constraint
Merits:

Provides clear, direct control over the amount of oil that is transshipped.
Ensures that transshipment does not exceed a specified threshold, aligning with explicit operational or strategic goals.
Suitable when there are clear operational reasons or business strategies to limit transshipment, such as cost, time, or quality considerations.

2. Modifying the Objective Function to Penalize Transshipment
Merits:

Offers more flexibility as it does not impose a hard limit but discourages excessive use of transshipment routes.
Allows the model to find a balance between cost optimization and reducing transshipment.
Suitable for scenarios where transshipment is not strictly limited, but there's a preference for direct routes.

If Can2Oil has a specific transshipment reduction target or faces operational constraints that necessitate limiting transshipment, the first approach (setting a transshipment limit constraint) is more appropriate. It provides a clear and enforceable limit on how much oil can be transshipped.

If the goal is more about generally favoring direct routes over transshipment without rigid constraints, then modifying the objective function to penalize transshipment offers a more balanced and flexible solution.

### question 6

In [60]:
import gurobipy as gp
from gurobipy import GRB

# Assuming you have the data loaded and the model setup as before

# Apply a discount to transportation costs for closer producers
discount_factor = 0.8  # e.g., 20% reduction in cost
for i in [1, 2, 3, 4, 5]:  # Assuming these are the IDs for North American producers
    for j in refinement_centers:
        cost_direct[i, j] *= discount_factor

# Reset the objective function with the updated costs
model.setObjective(
    gp.quicksum(cost_direct[i, j] * x[i, j] for i in direct_prod_facilities for j in refinement_centers) +
    gp.quicksum(cost_transship[i, k] * y[i, k] for i in transship_prod_facilities for k in transshipment_hubs) +
    gp.quicksum(cost_trans_to_refine[k, j] * z[k, j] for k in transshipment_hubs for j in refinement_centers),
    GRB.MINIMIZE)

# Solve the model with the updated costs
model.optimize()

# Output the optimal transportation cost
if model.status == GRB.OPTIMAL:
    print(f"Optimal Transportation Cost with Re-shoring: {model.objVal}")
else:
    print("Optimal solution was not found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.3.0 23D56)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 54 rows, 165 columns and 855 nonzeros
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+03]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.0706394e+30   2.600000e+31   2.070639e+00      0s
       7    2.1238557e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.123855655e+04
Optimal Transportation Cost with Re-shoring: 21238.556553760773


### question 7

The re-shoring model might still reduce transshipment if nearby producers offer competitive direct routes, but this is a byproduct of its primary goal of favoring closer sources.